# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [1]:
from models.arcs import generate, GenPhiloTextA
from utilities.loaders import load_lookup_table, load_hyper_params
from utilities.preprocessors import map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

# Load saved hyper parameters and lookup table

In [2]:
chars = load_lookup_table('./saved/misc/char_to_idx')
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [3]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Load same hyper params used in training model

In [4]:
hyper_params = load_hyper_params('./saved/misc/hyper_params.json')
hyper_params

{'d': 'notes',
 'model': 'GenPhiloTextA',
 'emb_dim': 256,
 'n_a': 512,
 'T_x': 100,
 'dense_layers_dims': [],
 'batch_size': 128,
 'alpha': 0.001,
 'lambda_': 0.8,
 'drop_prob': 0.4,
 'n_epochs': 100,
 'normalize': False,
 'n_unique': 57}

# Declare sample input to build inference model and access .summary()

In [5]:
sample_input = tf.random.uniform(shape=(1, hyper_params['T_x']), minval=0, maxval=hyper_params['n_unique'] - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, hyper_params['n_a']))
sample_c = tf.zeros(shape=(1, hyper_params['n_a']))

NameError: name 'T_x' is not defined

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [ ]:
saved_model = GenPhiloTextA(
    emb_dim=hyper_params['emb_dim'], 
    n_a=hyper_params['n_a'],
    n_unique=hyper_params['n_unique'],
    dense_layers_dims=hyper_params['dense_layers_dims'] + [hyper_params['n_unique']],
    drop_prob=hyper_params['drop_prob'],
    normalize=hyper_params['normalize'])
saved_model(sample_input)

<tf.Tensor: shape=(1, 100, 57), dtype=float32, numpy=
array([[[ 5.5873678e-03, -2.6368350e-04,  1.5447137e-03, ...,
         -3.2698223e-03,  7.5075077e-03,  3.5453830e-03],
        [ 1.1616457e-02,  6.7592282e-03,  6.1894311e-03, ...,
         -4.9574659e-03, -6.5224087e-03,  2.8680037e-03],
        [ 9.2312768e-03,  2.9715165e-03,  8.1195887e-03, ...,
         -2.1138368e-04, -8.3770920e-03,  7.4150846e-03],
        ...,
        [ 8.2587190e-03, -3.7701116e-03,  9.2248386e-03, ...,
          4.3318938e-03, -1.8714750e-03,  6.0311635e-03],
        [ 9.7461762e-03, -5.2710697e-03,  1.0019232e-03, ...,
          8.2420409e-03,  1.6472638e-03,  6.0437135e-03],
        [ 1.5433846e-02, -6.0290508e-03, -5.1153698e-03, ...,
          4.2357277e-03, -3.5584904e-05, -2.5300188e-03]]], dtype=float32)>

In [ ]:
sample_input = tf.random.uniform(shape=(1, hyper_params['T_x'] + 10), minval=0, maxval=hyper_params['n_unique'] - 1, dtype=tf.int32)
saved_model(sample_input)

<tf.Tensor: shape=(1, 110, 57), dtype=float32, numpy=
array([[[ 0.002675  , -0.00086119, -0.00047778, ...,  0.00191421,
          0.00229491,  0.00388378],
        [ 0.00456509, -0.00216243, -0.00619371, ...,  0.0066715 ,
          0.00487578,  0.00324689],
        [ 0.00195797, -0.00166749, -0.00855747, ...,  0.00636982,
          0.00572173, -0.0017583 ],
        ...,
        [ 0.00117389,  0.00187142, -0.00658506, ...,  0.01087044,
          0.00901425, -0.00111266],
        [-0.00165307, -0.00030379,  0.0010411 , ...,  0.0068941 ,
          0.00550996, -0.00089084],
        [ 0.00240167, -0.00169245,  0.00776001, ...,  0.007586  ,
          0.01555683, -0.00714594]]], dtype=float32)>

In [ ]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[ 0.01112122,  0.0199299 , -0.03027483, ...,  0.01302237,
         -0.01468526, -0.04417178],
        [-0.01288326, -0.02460574, -0.01761436, ..., -0.02062943,
          0.0130476 , -0.04972954],
        [-0.0159489 , -0.03664137, -0.0259202 , ...,  0.00595887,
          0.02686986, -0.04758003],
        ...,
        [-0.00363203,  0.02765051, -0.04040091, ..., -0.00243114,
         -0.01295518, -0.0447939 ],
        [ 0.01999731, -0.04859842,  0.00487623, ..., -0.00533441,
          0.03822137,  0.021401  ],
        [ 0.00241137,  0.03592468,  0.03221055, ..., -0.02678787,
          0.00648345, -0.00394202]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[-0.0206161 ,  0.04051947, -0.01342232, ..., -0.04582021,
          0.00831721, -0.01419644],
        [-0.04418119, -0.02083343, -0.00616296, ...

In [ ]:
saved_model.summary()

Model: "gen_philo_text_a"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 character-lookup (Embeddin  multiple                  14592     
 g)                                                              
                                                                 
 lstm-layer (LSTM)           multiple                  1574912   
                                                                 
 dense-layer-0 (Dense)       multiple                  29241     
                                                                 
Total params: 1618745 (6.18 MB)
Trainable params: 1618745 (6.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_a_100_3.0299.h5')

In [ ]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[-2.27964771e-31, -1.04065716e-31,  6.76143058e-32, ...,
          5.78990514e-33, -6.32452695e-35,  1.94203334e-32],
        [-2.33196533e-06, -5.57359442e-07,  7.22897767e-07, ...,
          3.29896963e-08,  4.24313072e-07, -7.51945777e-08],
        [-6.22333800e-06, -8.55494363e-06,  1.20172226e-05, ...,
          3.50521004e-05,  2.60806992e-05, -1.78583414e-05],
        ...,
        [ 1.16941873e-11,  5.07066784e-11,  2.42132052e-12, ...,
          1.95506927e-11, -9.94128564e-12,  5.03835099e-11],
        [-1.29197097e-09, -5.15809404e-08,  6.50408394e-09, ...,
          1.08325215e-08,  1.27126455e-07, -2.77360197e-08],
        [ 6.44838792e-07, -1.91505521e-07,  9.48950856e-08, ...,
          1.13875785e-06,  8.59064301e-07, -3.50037581e-08]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[

# generate characters using prompt

In [ ]:
prompts = ["hello"]
pred_ids_a = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=1.0)
pred_ids_b = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.8)
pred_ids_c = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.6)
pred_ids_d = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.4)


tf.Tensor([[32 29 36 36 39]], shape=(1, 5), dtype=int64)
tf.Tensor(
[[[-9.7952789e-01 -4.9550407e-02  7.8760695e-01 -9.4414473e-01
   -9.1001999e-01 -9.8696762e-01 -1.0040931e+00 -9.8399913e-01
   -9.6389389e-01 -1.4195003e-03 -9.9328291e-01 -3.3886015e-01
   -9.5995611e-01 -9.6912700e-01 -9.7229034e-01 -1.0030506e+00
   -9.4810855e-01 -1.0059516e+00 -9.9453384e-01 -9.9514633e-01
   -9.7885209e-01 -1.0189408e+00 -9.2216438e-01 -9.6369439e-01
   -9.5581770e-01  9.0898013e-01  3.4355694e-01  6.2924385e-01
    8.2376647e-01  9.0805137e-01  7.2464496e-01  5.3681183e-01
    9.0523934e-01  8.8704902e-01 -9.1242909e-01 -4.0618968e-01
    8.0495793e-01  6.1087257e-01  8.9836323e-01  9.1019720e-01
    2.1426669e-01 -9.6541142e-01  8.5857725e-01  9.0976363e-01
    9.2935693e-01  7.0334429e-01  2.4025665e-01  5.6967038e-01
   -9.3542272e-01  5.4357767e-01 -9.6410286e-01 -9.4257683e-01
   -9.8725337e-01 -9.6333629e-01 -1.0216787e+00 -9.2859411e-01
   -9.4580013e-01]
  [-1.1025574e+00 -2.9315751e-

In [ ]:
pred_ids_a

<tf.Tensor: shape=(1, 255), dtype=int64, numpy=
array([[32, 29, 36, 36, 39, 29, 43, 43, 39, 43,  2, 29, 38, 32, 32,  2,
        29, 33, 44,  2, 39, 30, 43, 31,  2, 38, 25, 42, 28,  2, 29, 33,
        28, 39, 38, 38, 45, 43,  2, 29, 25, 38,  9, 44, 39, 38, 43, 32,
        44, 36, 14, 33, 29, 38,  2, 29,  2, 40, 26,  2, 25, 38, 27, 39,
        49, 38, 39, 33, 33, 32, 46,  4, 38, 25, 42, 37, 33,  2, 17, 29,
        27, 43, 29,  2, 29, 33,  2, 27, 38, 39, 41,  2, 32, 44, 29, 44,
        43, 38, 29, 44, 44,  2, 42, 29, 22,  2, 32,  2, 32, 33,  2, 33,
        30, 46, 43, 45, 38,  2, 42, 42, 28, 25, 43, 38, 33, 39, 29, 33,
        43, 36, 44, 38, 42, 38,  2,  2,  2, 25, 33, 29, 51, 48,  2, 33,
        39, 29,  3, 33, 29, 25,  2, 25, 46, 45, 37, 39, 54, 28, 18, 51,
        48, 38, 46, 33, 44, 53, 44, 30,  2, 29, 29, 44, 38,  2, 54, 11,
        36, 45, 25,  2, 29,  1, 20, 38,  2, 29, 28, 11,  2, 40, 36,  2,
        31, 43, 43, 47, 32,  6, 49, 27, 28, 43,  2, 43,  9,  2, 33, 43,
        48, 45, 

# Extract trainable layers

In [ ]:
decode_predictions(pred_ids_a, idx_to_char)

'helloessos enhh eit ofsg nard eidonnus ean,tonshtl1ien e pb ancoynoiihv"narmi 4ecse ei cnoq htetsnett re: h hi ifvsun rrdasnioeisltnrn   aie´x ioe!iea avumoïd6´xnvitétf eetn ï.lua e\n8n ed. pl gsswh\'ycds s, isxu.oo …ihn hu  p\npesven"tagfo-fsfanigdne! nnaaa'

In [ ]:
decode_predictions(pred_ids_b, idx_to_char)

'hello:al w ny aar dofloeheah lw6irltfih tas ieeewstacaatrn8h n fi9uyna  eoc cnm i e l i etns aeas a edif\not t7ny  eanc uishgmhtaee\ni a´n.ede heu  swato lw oleit e eoah"ac  thoaon tnnv t ou  te eode.aeemaarho hp adni eetztzto aa6oeem ! ne   s nn  uiisho is'

In [ ]:
decode_predictions(pred_ids_c, idx_to_char)

'hellon eiheoahilsyet  e   h eenn anehio -  s emcwtahaltntn aseehw   is anos ar     net   u a  etnf te   e!se eo senwl u b   rna  esieeid etttliee is o  ao nhn hn soteoesh te?a\net  ecta s n r aoatt tai y  tfanwbf  dyasoih evo ds  o  vee  çe oihha t tma  o '

In [ ]:
decode_predictions(pred_ids_d, idx_to_char)

'hello     f   ee  e h t o  n  eo  r  ot ea    a a en ec  eiagea n a   o   e en      ara  i e ed   n     t oe     ae o a e a   at   it      ele e    e  e ei ee a th ne   h     et  l i  d  ere  ot   a     t   n   to       cto    oteehe a   oe l      i   ie '